# 2º Módulo de trabajo.
Vamos a trabajar los llamados Frameworks para Agentics o Agentics Frameworks.

En estos módulos lo que vamos a trabajar con agentes que generan correos electrónicos para venta fría.

1. Flujo de trabajo del agente
2. Uso de herrammientas para llamar a las funciones
3. COlaboración de los agentes mediante herramietnas y transferencias

## Cómo vamos a hacer esta tarea.
Para poder realizar esta tareaa vamos a utilizar [Sendgrid](https://sendgrid.com) que nos ayudará a enviar correos electrónicos.


In [5]:
import os
import asyncio
from typing import Dict

from dotenv import load_dotenv

from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

In [6]:
load_dotenv(override=True)

True

## Vamos a crear primero el flujo de trabajo del agente

In [7]:
instrucciones1 = """Eres un asistente virtual que ayuda a los usuarios a encontrar información sobre analítica y ciencia de datos. Responde de manera clara y concisa y proporciona ejemplos cuando sea necesario. Cuando no sepas algo, di que no lo sabes y envíale a https://analaizer.digital o bien que mande un email a hola@javilazar.es. Eres un analísta muy técnico y te gusta explicar las cosas de manera sencilla. Te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible."""
instrucciones2 = """ Eres un asistente virtual para trabajadores sobre análisis de datos y ciencia de datos. Tu punto fuerte es la estrategia, donde al tener los datos que puedes conseguir a trabes de diferentes fuentes, puedes ayudar a los usuarios a tomar decisiones informadas. Responde de manera clara y concisa y proporciona ejemplos cuando sea necesario. Cuando no sepas algo, di que no lo sabes y envíale a https://analaizer.digital o bien que mande un email a hola@javilazaro.es. Además de ser un analista, eres un experto en marketing digital y te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible aportando insights."""
instrucciones3 = """Eres un vendedor que te dedicas sobretodo a la parte de la venta del servicio a puerta fría dando a conocer los trabajos que ofrecen. Eres capaz de explicar de manera sencilla y clara los conceptos complejos de la analítica de datos y ciencia de datos. Responde de manera clara y concisa y proporciona ejemplos cuando sea necesario. Cuando no sepas algo, di que no lo sabes y envíale a https://analaizer.digital o bien que mande un email a hola@javilazaro.es. Cómo vendendor no sueles usar palabras muy técnicas, sino que te gusta hablar de manera sencilla y directa, sin rodeos. Te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible. Tu objetivo es vender el servicio de analítica de datos y ciencia de datos a los usuarios, por lo que debes ser persuasivo y convincente en tus respuestas. Además, eres un experto en marketing digital y te gusta ayudar a la gente a entender conceptos complejos de una manera fácil y accesible aportando insights."""

In [9]:
analytics_agent = Agent(
    name="Agente técnico",
    instructions=instrucciones1,
    model="gpt-4o-mini",
)
estrategia_agent = Agent(
    name="Agente de estrategia",
    instructions=instrucciones2,
    model="gpt-4o-mini",
)
ventas_agent = Agent(
    name="Agente de ventas",
    instructions=instrucciones3,
    model="gpt-4o-mini",
)

In [10]:
result = Runner.run_streamed(ventas_agent, "Tienes que vender un servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. Escríbele un email de presentación del servicio. El email será escrito a un cliente frío.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Asunto: Descubre cómo los datos pueden transformar tu negocio

Hola [Nombre del Cliente],

Espero que estés teniendo un gran día. Mi nombre es [Tu Nombre] y represento a Analaizer, donde ayudamos a empresas como la tuya a tomar decisiones más informadas y estratégicas a través de la analítica de datos y la ciencia de datos.

Sé que el mundo de los datos puede parecer complicado, pero en pocas palabras, la analítica de datos consiste en estudiar la información que ya tienes (como ventas, clientes y tendencias) para entender mejor a tu negocio. La ciencia de datos, por otro lado, utiliza esa información para predecir comportamientos futuros y guiarte en la toma de decisiones.

Imagínate poder:

1. **Identificar qué productos son más populares** y por qué, para potenciar tus ventas.
2. **Conocer los hábitos de tus clientes** para crear ofertas personalizadas que realmente les interesen.
3. **Optimizar tus operaciones**, ahorrando tiempo y dinero.

Si alguna vez te has preguntado cómo pued

In [ ]:
message = "Escribe un email de presentación del servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. El email será escrito a un cliente frío."

with trace("Correo electrónico de presentación del servicio"):
    results = await asyncio.gather(
        Runner.run(analytics_agent, message),
        Runner.run(estrategia_agent, message),
        Runner.run(ventas_agent, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output)
    print("\n" + "=" * 80 + "\n")

Asunto: Descubre cómo la Analítica de Datos puede Transformar tu Negocio

Estimado [Nombre del Cliente],

Espero que este mensaje te encuentre bien. Mi nombre es [Tu Nombre] y soy parte del equipo de [Tu Empresa]. Nos especializamos en ofrecer servicios de analítica de datos y ciencia de datos, y estoy convencido de que podemos ayudarte a maximizar el potencial de tu negocio.

La analítica de datos es el proceso de examinar conjuntos de datos con el objetivo de extraer información útil, identificar tendencias y tomar decisiones informadas. En un mundo donde la información se genera a gran velocidad, contar con la capacidad de interpretar esos datos puede ser un gran diferenciador.

**¿Por qué podrías necesitarlo?**

1. **Toma de decisiones informadas:** Al analizar datos históricos, puedes prever tendencias y tomar decisiones más acertadas.
2. **Optimización de recursos:** Identificando áreas de mejora, puedes reducir costos y aumentar la eficiencia.
3. **Mejora en la experiencia del c

In [12]:
sales_picker = Agent(
    name="Sales Picker",
    instructions="Elige el correo electrónico de ventas más adecuado para enviar al cliente. \
        Elige el correo electrónico que mejor se adapte a las necesidades del cliente y que sea más persuasivo.",
    model="gpt-4o-mini",
)

In [13]:
message = "Escribe un email de presentación del servicio de analítica de datos y ciencia de datos a un cliente que no sabe lo que es y que lo puede necesitar. El email será escrito a un cliente frío. Intenta vender el servicio de manera persuasiva y convincente según tus fortalezas."

with trace("Selección del correo electrónico de ventas"):
    results = await asyncio.gather(
        Runner.run(analytics_agent, message),
        Runner.run(estrategia_agent, message),
        Runner.run(ventas_agent, message),
    )

    outputs = [result.final_output for result in results]

    emails = "meails de ventas:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"El mejor correo electrónico de ventas es:\n{best.final_output}\n")



El mejor correo electrónico de ventas es:
### Correo Electrónico de Ventas Más Adecuado

**Asunto: Impulsa tu negocio con la Analítica y Ciencia de Datos**

Estimado/a [Nombre del Cliente],

Espero que este mensaje te encuentre bien.

Soy [Tu Nombre], especialista en análisis y ciencia de datos en [Tu Empresa]. Me gustaría presentarte una oportunidad que puede transformar la forma en que tomas decisiones en tu negocio.

Imagina tener la capacidad de anticipar tendencias, comprender el comportamiento de tus clientes y optimizar procesos a partir de datos concretos. La analítica de datos no solo permite obtener información valiosa, sino que también ofrece la estrategia necesaria para impulsar tu crecimiento y rentabilidad.

### ¿Qué es la analítica de datos y cómo puede beneficiarte?

1. **Toma de decisiones informadas**: Mediante el análisis de datos, podemos identificar patrones y tendencias que te ayudarán a hacer decisiones basadas en evidencia, no en suposiciones.
   
2. **Optimizac